<a href="https://colab.research.google.com/github/charlieyang1557/Yutian-Yang/blob/master/Superconductivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

supercon = pd.read_csv("train.csv")
supercon

FileNotFoundError: ignored

In [0]:
# Compute the correlation matrix
corr = supercon.corr()
variables = corr["critical_temp"].abs().sort_values(ascending = False).head(16)

variable_names = []
for index, val in variables.iteritems():
  variable_names.append(index)


supercon_features = supercon[variable_names]
supercon_features

NameError: ignored

In [0]:
corr = supercon_features.corr()
new_corr = corr.abs().mean(axis=0)
new_corr.sort_values(ascending=False)

# range_atomic_radius has the highest correlation with critical temp and other features
# the feature atomic radius and Valence seem important because it appears multiple times


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, annot=True)
plt.show()

# wtd_std_ThermalConductivity has the highest postivie correlation with critical temp,
# wtd_mean_valence has the highest negative correlation with critical temp
#

In [0]:
sns.boxplot(x="number_of_elements", y="critical_temp", data=supercon_features,  dodge=False)

# general trend: if more number of elements, then higher critical temp

In [0]:
sns.pairplot(x_vars=["number_of_elements"], y_vars=["critical_temp"], data=supercon_features, size=7)

from numpy.polynomial.polynomial import polyfit
b, m = polyfit(supercon_features['number_of_elements'], supercon_features['critical_temp'], 1)

plt.plot(supercon_features['number_of_elements'], supercon_features['critical_temp'], '.')
plt.plot(supercon_features['number_of_elements'], b + m * supercon_features['number_of_elements'], '-')
plt.title('number_of_elements vs. critical_temp')
plt.xlabel('number_of_elements')
plt.ylabel('critical_temp')
plt.show()

In [0]:

sns.pairplot(x_vars=["wtd_std_ThermalConductivity"], y_vars=["critical_temp"], data=supercon_features, size=7)

from numpy.polynomial.polynomial import polyfit
b, m = polyfit(supercon_features['wtd_std_ThermalConductivity'], supercon_features['critical_temp'], 1)

plt.plot(supercon_features['wtd_std_ThermalConductivity'], supercon_features['critical_temp'], '.')
plt.plot(supercon_features['wtd_std_ThermalConductivity'], b + m * supercon_features['wtd_std_ThermalConductivity'], '-')
plt.title('wtd_std_ThermalConductivity vs. critical_temp')
plt.xlabel('wtd_std_ThermalConductivity')
plt.ylabel('critical_temp')
plt.show()

In [0]:
#fix later
sns.pairplot(x_vars=["wtd_mean_Valence"], y_vars=["critical_temp"], data=supercon_features, size=7)

from numpy.polynomial.polynomial import polyfit
b, m = polyfit(supercon_features['wtd_mean_Valence'], supercon_features['critical_temp'], 1)

plt.plot(supercon_features['wtd_mean_Valence'], supercon_features['critical_temp'], '.')
plt.plot(supercon_features['wtd_mean_Valence'], b + m * supercon_features['wtd_mean_Valence'], '-')
plt.title('wtd_mean_Valence vs. critical_temp')
plt.xlabel('wtd_mean_Valence')
plt.ylabel('critical_temp')
plt.show()

In [0]:
from sklearn import metrics

mutual_conditional_entropies = np.zeros((16,16))

for i in range(len(supercon_features.columns)):
  ith_row = []
  for j in range(len(supercon_features.columns)):

    #conditional entropy:
    x_given_y = metrics.mutual_info_score(supercon_features.iloc[:,i],supercon_features.iloc[:,i]) - metrics.mutual_info_score(supercon_features.iloc[:,i],supercon_features.iloc[:,j])
    y_given_x = metrics.mutual_info_score(supercon_features.iloc[:,j],supercon_features.iloc[:,j]) - metrics.mutual_info_score(supercon_features.iloc[:,j],supercon_features.iloc[:,i])

    #original entropy:
    x_original_entropy = metrics.mutual_info_score(supercon_features.iloc[:,i],supercon_features.iloc[:,i])
    y_original_entropy = metrics.mutual_info_score(supercon_features.iloc[:,j],supercon_features.iloc[:,j])

    #mutual conditional entropy:
    mutual_conditional_entropy = 0.5*(x_given_y/x_original_entropy) + 0.5*(y_given_x/y_original_entropy)
    #append to ith row
    ith_row.append(mutual_conditional_entropy)
  mutual_conditional_entropies[i,:] = ith_row
  

In [0]:
  mutual_conditional_entropies


In [0]:
plt.figure(figsize = (11,9))
sns.heatmap(mutual_conditional_entropies, xticklabels= supercon_features.columns, yticklabels=supercon_features.columns,annot=True)
plt.show()

#similar to correlation heatmap,
# wtd_std_ThermalConductivity - 0.22
# wtd_entropy_atomic_mass - 0.2
# wtd_entropy_atomic_radius - 0.2
# wtd_std_atomic_radius - 0.21
# wtd_std_fie - 0.2

In [0]:
#manually categorize critical temp into three levels: low, mid, high

cut_1 = supercon.iloc[:,81].mean() - supercon.iloc[:,81].std()
cut_2 = supercon.iloc[:,81].mean() + supercon.iloc[:,81].std()


bins = [0, cut_1, cut_2,200]
group_names = [0,1,2]
supercon['critical_temp_level'] = pd.cut(supercon['critical_temp'], bins, labels=group_names)


In [0]:
# regression analysis

supercon_features = supercon_features.assign(critical_temp_level = supercon.iloc[:,82])

X = supercon_features.iloc[:, [1,2, 3,4,5,6,7,8,9,10,11,12,13,14,15]].to_numpy().astype('float') #divide the data for classification purpose
Y = supercon_features.iloc[:, 0].to_numpy().astype('int')

# random sampling
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size=0.3, random_state=10)

In [0]:
# Use LOOCV to classify species (1-dim)

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
import sklearn.linear_model as skl_lm

# LeaveOneOut CV linear regression

regr = skl_lm.LinearRegression()
loo = LeaveOneOut()
loo.get_n_splits(supercon_features)

regr.fit(X_train, Y_train)
scores = list()

# The mean square error
np.mean((regr.predict(X_test) - Y_test)**2)

# Explained variance score: 1 is perfect prediction and 0 means that there is no linear relationship between X and y.
score = cross_val_score(regr, X_test, Y_test, cv=loo, scoring='neg_mean_squared_error').mean()

scores.append(-score)
scores

In [0]:

model = regr.fit(X_train, Y_train)
prediction = regr.predict(X_test)
r_square_score = regr.score(X_test,Y_test)
plt.scatter(prediction,Y_test)
plt.xlabel("True Critical Temp")
plt.ylabel("Predicted Critical Temp")
plt.title("Predicted values Vs. True Values")
plt.show()
print("R square:" + str(r_square_score))

In [0]:
import random

X_train_poly = supercon_features["wtd_std_ThermalConductivity"].sample(frac = 0.7)
train_idx = X_train_poly.index.values
test_idx = []
for i in supercon_features.index:
  if i not in train_idx:
    test_idx.append(i)

X_test_poly = supercon_features.iloc[test_idx].iloc[:,1]

In [0]:
# polynomial regression

# Use LOOCV to classify species (1-dim)

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
import sklearn.linear_model as skl_lm

p_order = np.arange(1,11)
r_state = np.arange(0,10)

# LeaveOneOut CV
import random

regr = skl_lm.LinearRegression()
loo = LeaveOneOut()
loo.get_n_splits(supercon_features)
scores = list()




for i in p_order:
    poly = PolynomialFeatures(i)
    X_poly = poly.fit_transform(X_train_poly.values.reshape(-1,1))
    score = cross_val_score(regr, X_poly, Y_train, cv=loo, scoring='neg_mean_squared_error').mean()
    scores.append(score)

In [0]:
fig, (ax1) = plt.subplots(1, figsize=(10,4))

# Note: cross_val_score() method return negative values for the scores.
# https://github.com/scikit-learn/scikit-learn/issues/2439

ax1.plot(p_order, np.array(scores)*-1, '-o')
ax1.set_title('LOOCV')

for ax in fig.axes:
    ax.set_ylabel('Mean Squared Error')
    ax.set_xlabel('Degree of Polynomial')
    
# when degree of polynomial equals 5, it reaches the elbow point, which means the model will be most efficient at poly degree equals to 5.

In [0]:
regr = skl_lm.LinearRegression()
model = regr.fit(X_train_poly.values.reshape(-1,1), Y_train)
prediction = regr.predict(X_test_poly.values.reshape(-1,1))
r_square_score = regr.score(X_test_poly.values.reshape(-1,1),Y_test)
plt.scatter(prediction,Y_test)
plt.xlabel("True Critical Temp")
plt.ylabel("Predicted Critical Temp")
plt.title("Predicted values Vs. True Values")
plt.show()
print("R square:" + str(r_square_score))

In [0]:
# machine learning approach analysis

X = supercon_features.iloc[:, [1,2, 3,4,5,6,7,8,9,10,11,12,13,14,15]].to_numpy().astype('float') #divide the data for classification purpose
Y = supercon_features.iloc[:, 16].to_numpy().astype('int')

# random sampling
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size=0.3, random_state=10)

In [0]:
# Use Grid Search and Cross Validation to classify species (K-dim)

from sklearn.model_selection import StratifiedShuffleSplit

split=StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=100)
for train_index, test_index in split.split(X,Y):
    X_train_strat, Y_train_strat = X[train_index], Y[train_index]
    X_test_strat, Y_test_strat = X[test_index], Y[test_index]

In [0]:
import sklearn.linear_model as skl_lm
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV

param_grid=[{'n_estimators':[3, 10, 30], 'max_features':[1,2,3,4,5,6,7,8,9],'bootstrap':[True,False]}]
tree=RandomForestClassifier()
grid_search=GridSearchCV(tree, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train_strat, Y_train_strat)

# Use Random Forest as our classifier

In [0]:
grid_search.best_estimator_

In [0]:
result=pd.DataFrame(grid_search.cv_results_['params'])
result['accuracy']=grid_search.cv_results_['mean_test_score']
result.sort_values('accuracy', ascending=False)

In [0]:
from sklearn.metrics import accuracy_score

final_model=grid_search.best_estimator_
test_final_pred=final_model.predict(X_test_strat)
accuracy_score(Y_test_strat, test_final_pred)

In [0]:
from sklearn.metrics import confusion_matrix
test_final_pred=final_model.predict(X_test_strat)
conf=confusion_matrix(Y_test_strat, test_final_pred)
conf

In [0]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

classifier = grid_search.best_estimator_

titles_options = [("Confusion matrix", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test_strat, Y_test_strat,
                                 display_labels=set(supercon_features["critical_temp_level"].values),
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [0]:
def decision_region(X, Y, model, prob=None):
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    x1, x2 = np.meshgrid(np.arange(x1_min, x1_max, 0.1),
                         np.arange(x2_min, x2_max, 0.1))

    Z = model.predict(np.c_[x1.ravel(), x2.ravel()])
    
    if prob is not None:
        prob_max=model.predict_proba(np.c_[x1.ravel(), x2.ravel()]).max(1)
        Z[prob_max<prob]=-1
    
    Z = Z.reshape(x1.shape)

    plt.contourf(x1, x2, Z, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=Y, s=20, edgecolor='k')
    plt.show()

In [0]:

# Multiclass Regression (One vs Rest)
supercon_features_valence_fie=supercon_features.iloc[:, [1,15]].to_numpy().astype('float') #  wtd_std_fie and wtd_std_thermalconductivity from corrogram
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

logit1=OneVsRestClassifier(LogisticRegression(penalty = 'none'))
logit1.fit(supercon_features_valence_fie,Y)
decision_region(supercon_features_valence_fie,Y,logit1)

In [0]:
logit2=OneVsOneClassifier(LogisticRegression(penalty='none'))
logit2.fit(supercon_features_valence_fie,Y)
decision_region(supercon_features_valence_fie,Y,logit2)

In [0]:
#multinomial classification

logit3=LogisticRegression(penalty = 'none', multi_class='multinomial')
logit3.fit(supercon_features_valence_fie,Y)
decision_region(supercon_features_valence_fie,Y,logit3)

In [0]:
# svm
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [0]:
critical_temp_low =((supercon_features.iloc[:, 16].to_numpy().astype('int')) == 0).astype('int') # 1 if low critical temp else 0

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [10, 3]):
  scale=StandardScaler()
  X_scale=scale.fit_transform(supercon_features_valence_fie)
  model = SVC(kernel='linear', C=C).fit(X_scale, critical_temp_low)
  axi.scatter(X_scale[:, 0], X_scale[:, 1], c=critical_temp_low, s=50, cmap='autumn')
  plot_svc_decision_function(model, axi)
  axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
  axi.set_title('C = {0:.1f}'.format(C), size=14)
    
